In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

d:\conda_envs\nlp_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- 1. Muat Model dan Tokenizer yang Telah Disimpan ---
MODEL_DIR = './model_final'

try:
    # Muat tokenizer dan model dari direktori yang sama
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
except EnvironmentError:
    print(f"Direktori model '{MODEL_DIR}' tidak ditemukan. Pastikan Anda sudah menjalankan langkah penyimpanan sebelumnya.")
    exit()

In [3]:
# --- 2. Siapkan Device (GPU/CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model berhasil dimuat dan berjalan di device: {device}")


Model berhasil dimuat dan berjalan di device: cuda


In [4]:
# --- 3. Buat Fungsi untuk Prediksi ---
def predict(text):
    """Fungsi untuk memprediksi label dari sebuah teks."""
    # Set model ke mode evaluasi
    model.eval()

    # Tokenisasi teks input
    encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True,   # Menambahkan simbol khusus di awal dan akhir kalimat agar model bisa memahami strukturnya
      max_length=128, # Membatasi panjang kalimat agar maksimal 128 kata (token)        
      padding='max_length', # Jika kalimatnya terlalu pendek, akan ditambah kata kosong agar panjangnya sama
      truncation=True, # Jika kalimatnya terlalu panjang, sisanya akan dibuang agar tidak melebihi batas
      return_attention_mask=True,  # Memberi tanda pada bagian mana yang merupakan kalimat asli dan mana yang hanya tambahan
      return_tensors='pt',  # Mengubah hasil pemrosesan menjadi format khusus yang bisa dibaca oleh mesin (PyTorch)
    )

    # Pindahkan tensor ke device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Lakukan prediksi tanpa menghitung gradien
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        # Ambil prediksi dengan probabilitas tertinggi
        prediction_idx = torch.argmax(outputs.logits, dim=1).item()

    # Mapping indeks ke label yang bisa dibaca manusia
    label_map = {0: 'Bukan Ujaran Kebencian', 1: 'Ujaran Kebencian'}
    
    return label_map[prediction_idx]

In [ ]:
# --- 4. Buat Loop Interaktif ---
if __name__ == '__main__':
    print("\n--- Tes Model Klasifikasi Ujaran Kebencian ---")
    print("Ketik 'exit' untuk keluar.")
    
    while True:
        # Minta input dari pengguna
        user_input = input("\nMasukkan teks: ")

        if user_input.lower() == 'exit':
            print("Terima kasih! Program berhenti.")
            break
        
        # Lakukan prediksi dan cetak hasilnya bersama teks input
        prediction_result = predict(user_input)
        print(f"Teks: {user_input}")
        print(f"Prediksi: {prediction_result}")


--- Tes Model Klasifikasi Ujaran Kebencian ---
Ketik 'exit' untuk keluar.
Teks: jokowi buruk rupa
Prediksi: Ujaran Kebencian
Teks: jokowi jomok
Prediksi: Bukan Ujaran Kebencian
Teks: madara ganteng dan melihara anjing
Prediksi: Bukan Ujaran Kebencian
Teks: madara anjing
Prediksi: Ujaran Kebencian


In [2]:
!pip install lime


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/12.9 MB 4.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.6/12.9 MB 4.1 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/12.9 MB 4.2 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/12.9 MB 4.2 MB/s eta 0:00:03
   ---------------- ----------------------- 5.2/12.9 MB 4.2 MB/s eta 0:00:02
   ------------------ --------------------- 6.0/12.9 MB 4.2 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/12.9 MB 4.2 MB/s eta 0:00:02
   ------------------------ --------------- 7.9/12.9 MB 4.3 MB/s eta 0:00:02
   -------------------------- ------------- 8.7/12.9 MB 4.2 MB/s eta 0:00:02
   ----------------------------- ---------- 9.4/12.9 MB 4.

  DEPRECATION: Building 'lime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'lime'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# ============================================================
# 🧠 MODEL INDOBERT + CETAK KATA YANG MEMICU (LIME EXPLANATION)
# ============================================================

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

# Coba impor LIME, kalau belum ada langsung install
try:
    from lime.lime_text import LimeTextExplainer
except ImportError:
    import os
    os.system('pip install -q lime')
    from lime.lime_text import LimeTextExplainer

# --- 1. Muat Model dan Tokenizer ---
MODEL_DIR = './model_final'
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
except EnvironmentError:
    print(f"Direktori model '{MODEL_DIR}' tidak ditemukan. Pastikan sudah disimpan sebelumnya.")
    exit()

# --- 2. Siapkan Device (GPU/CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model berhasil dimuat dan berjalan di device: {device}")

# --- 3. Fungsi Prediksi Probabilitas (untuk LIME) ---
def predict_proba(texts):
    model.eval()
    all_probs = []
    for text in texts:
        encoding = tokenizer(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(device)

        with torch.no_grad():
            outputs = model(**encoding)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1)
            all_probs.append(probs.cpu().numpy().flatten())
    return np.array(all_probs)

# --- 4. Fungsi Prediksi Label ---
def predict_label(text):
    probs = predict_proba([text])[0]
    label_idx = np.argmax(probs)
    label_map = {0: 'Bukan Ujaran Kebencian', 1: 'Ujaran Kebencian'}
    return label_map[label_idx], probs

# --- 5. Fungsi Interpretasi dan Ekstraksi Kata Kasar ---
def get_trigger_words(text, threshold=0.05):
    """Menemukan kata yang paling memicu model mendeteksi ujaran kebencian"""
    explainer = LimeTextExplainer(class_names=['Non Hate', 'Hate'])
    exp = explainer.explain_instance(text, predict_proba, num_features=10)
    
    # Ambil kata yang bobotnya positif (kontribusi ke kelas Hate)
    trigger_words = [word for word, weight in exp.as_list() if weight > threshold]
    return trigger_words, exp

# --- 6. Loop Interaktif ---
if __name__ == '__main__':
    print("\n--- Tes Model Klasifikasi Ujaran Kebencian + Deteksi Kata Pemicu ---")
    print("Ketik 'exit' untuk keluar.\n")

    while True:
        user_input = input("Masukkan teks: ").strip()

        # Jika kosong
        if not user_input:
            print("⚠️  Harap masukkan teks terlebih dahulu.\n")
            continue

        if user_input.lower() == 'exit':
            print("Terima kasih! Program dihentikan.")
            break

        label, probs = predict_label(user_input)
        print(f"\nTeks: {user_input}")
        print(f"Prediksi: {label}")
        print(f"Probabilitas [Non Hate, Hate]: {np.round(probs, 3)}")

        # Dapatkan kata-kata pemicu
        trigger_words, exp = get_trigger_words(user_input)

        if trigger_words:
            print(f"Kata yang memicu deteksi hate speech: {trigger_words}")
        else:
            print("Tidak ditemukan kata pemicu hate speech yang signifikan.")

        print("\n" + "="*60 + "\n")

d:\conda_envs\nlp_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model berhasil dimuat dan berjalan di device: cuda

--- Tes Model Klasifikasi Ujaran Kebencian + Deteksi Kata Pemicu ---
Ketik 'exit' untuk keluar.

